In [ ]:
%cd ../

In [ ]:
import pandas as pd

In [ ]:
def create_table(res_file):
    df = pd.read_csv(res_file, index_col=None)
    df = df[["model", "measure", "value", "task"]]
        
    # Replace measure and model names
    measure_map = {
        "doc_sim": "DocSim", 
        "max_iou": "\maxiou{}", 
        "latsim": "\ourmeasure{}", 
        "mean_iou": "MeanIoU", 
        }
    df.measure = df.measure.map(measure_map)
    model_map = {"bart": "BART", "blt": "BLT", "vqdiffusion": "VQDiffusion", "layoutdm": "LayoutDM", "maskgit": "MaskGIT"}
    df.model = df.model.map(model_map)

    # for each ("measure", "model"), compute mean
    summary = {"measure": [], "model": [], "mean": [], "task": []}
    for task in df.task.unique():
        for model in df.model.unique():
            for measure in df.measure.unique():
                value = df[(df.task == task) & (df.model == model) & (df.measure == measure)].value
                summary["measure"].append(measure)
                summary["model"].append(model)
                summary["mean"].append(value.mean())
                summary["task"].append(task)

    summary = pd.DataFrame(summary)
    summaries = {}
    for task in summary.task.unique():
        sub_summary = summary[summary.task == task]
        sub_summary = sub_summary.pivot(index="model", columns="measure", values="mean")
        # x100 for visibility
        sub_summary = sub_summary * 100
        summaries[task] = sub_summary

    return summaries

def load_table_uncond(res_file):
    df = pd.read_csv(res_file, index_col=None)
    df = df[["model", "measure", "value"]]
    measure_map = {"doc_sim": "DocSim", "max_iou": "\maxiou{}", "mmd": "\ourmeasure{}-MMD", "mean_iou": "MeanIoU", "fid": "FID"}
    df.measure = df.measure.map(measure_map)
    model_map = {"bart": "BART", "blt": "BLT", "vqdiffusion": "VQDiffusion", "layoutdm": "LayoutDM", "maskgit": "MaskGIT"}
    df.model = df.model.map(model_map)
    # x100 for max_iou and latsim
    df.loc[df.measure.isin(["\maxiou{}", "\ourmeasure{}-MMD"]), "value"] *= 100

    # for each ("measure", "model"), compute mean
    summary = {"measure": [], "model": [], "mean": []}
    for model in df.model.unique():
        for measure in df.measure.unique():
            mean = df[(df.model == model) & (df.measure == measure)]["value"].mean()
            summary["measure"].append(measure)
            summary["model"].append(model)
            summary["mean"].append(mean)

    summary = pd.DataFrame(summary)
    summary = summary.pivot(index="model", columns="measure", values="mean")
    # reorder columns
    summary = summary[["\maxiou{}", "FID", "\ourmeasure{}-MMD"]]
    return summary

# Create a table for unconditional & conditional generation

In [ ]:
rico_cond= create_table("data/results/eval_conditional/rico/result.csv")["c"]
publay_cond= create_table("data/results/eval_conditional/publaynet/result_c.csv")["c"]
res_cond = {"textsc{RICO}": rico_cond, "textsc{PubLayNet}": publay_cond}
res_cond = pd.concat(res_cond, axis=0)

rico_cond_rank = rico_cond.rank(axis=0, ascending=True)
publay_cond_rank = publay_cond.rank(axis=0, ascending=True)
cond_rank = {"textsc{RICO}": rico_cond_rank, "textsc{PubLayNet}": publay_cond_rank}
cond_rank = pd.concat(cond_rank, axis=0)

rico_uncond = load_table_uncond("data/results/eval_unconditional/rico_results.csv")
publay_uncond = load_table_uncond("data/results/eval_unconditional/publaynet_results.csv")
res_uncond = {"textsc{RICO}": rico_uncond, "textsc{PubLayNet}": publay_uncond}
res_uncond = pd.concat(res_uncond, axis=0)

rico_uncond_rank = pd.concat((rico_uncond[["\maxiou{}"]].rank(axis=0, ascending=True), rico_uncond[["\ourmeasure{}-MMD", "FID"]].rank(axis=0, ascending=False)), axis=1)
publay_uncond_rank = pd.concat((publay_uncond[["\maxiou{}"]].rank(axis=0, ascending=True), publay_uncond[["\ourmeasure{}-MMD", "FID"]].rank(axis=0, ascending=False)), axis=1)
uncond_rank = {"textsc{RICO}": rico_uncond_rank, "textsc{PubLayNet}": publay_uncond_rank}
uncond_rank = pd.concat(uncond_rank, axis=0)

rank = {"Label-conditioned Gen.": cond_rank, "Unconditional Gen.": uncond_rank}
rank = pd.concat(rank, axis=1)

table = pd.concat({"Unconditional Gen.": res_uncond, "Label-conditioned Gen.": res_cond}, axis=1)
table = table[["Unconditional Gen.", "Label-conditioned Gen."]]

styler = table.style\
.background_gradient(cmap="Blues", gmap=rank, axis=None, high=0.5)\
.format("{:.2f}")
# table.to_latex("tbl/eval_generated_summary.tex", float_format="%.2f")
styler.to_latex("tbl/eval_generated_summary.tex", convert_css=True, hrules=True, position_float="centering")
styler

# Full results (in supplement)

In [ ]:
rico_cond= create_table("data/results/eval_conditional/rico/result.csv")["c"]
publay_cond= create_table("data/results/eval_conditional/publaynet/result_c.csv")["c"]
res_cond = {"textsc{RICO}": rico_cond, "textsc{PubLayNet}": publay_cond}
res_cond = pd.concat(res_cond, axis=0)

rico_cond_rank = rico_cond.rank(axis=0, ascending=True)
publay_cond_rank = publay_cond.rank(axis=0, ascending=True)
cond_rank = {"textsc{RICO}": rico_cond_rank, "textsc{PubLayNet}": publay_cond_rank}
cond_rank = pd.concat(cond_rank, axis=0)

rico_uncond = load_table_uncond("data/results/eval_unconditional/rico_results.csv")
publay_uncond = load_table_uncond("data/results/eval_unconditional/publaynet_results.csv")
res_uncond = {"textsc{RICO}": rico_uncond, "textsc{PubLayNet}": publay_uncond}
res_uncond = pd.concat(res_uncond, axis=0)

rico_uncond_rank = pd.concat((rico_uncond[["\maxiou{}"]].rank(axis=0, ascending=True), rico_uncond[["\ourmeasure{}-MMD", "FID"]].rank(axis=0, ascending=False)), axis=1)
publay_uncond_rank = pd.concat((publay_uncond[["\maxiou{}"]].rank(axis=0, ascending=True), publay_uncond[["\ourmeasure{}-MMD", "FID"]].rank(axis=0, ascending=False)), axis=1)
uncond_rank = {"textsc{RICO}": rico_uncond_rank, "textsc{PubLayNet}": publay_uncond_rank}
uncond_rank = pd.concat(uncond_rank, axis=0)

rico_completion= create_table("data/results/eval_conditional/rico/result.csv")["partial"]
publay_completion= create_table("data/results/eval_conditional/publaynet/partial_result.csv")["partial"]
res_completion = {"textsc{RICO}": rico_completion, "textsc{PubLayNet}": publay_completion}
res_completion = pd.concat(res_completion, axis=0)

rico_comp_rank = rico_completion.rank(axis=0, ascending=True)
publay_comp_rank = publay_completion.rank(axis=0, ascending=True)
comp_rank = {"textsc{RICO}": rico_comp_rank, "textsc{PubLayNet}": publay_comp_rank}
comp_rank = pd.concat(comp_rank, axis=0)

rank = {"Label-conditioned Gen.": cond_rank, "Completion Gen.": comp_rank, "Unconditional Gen.": uncond_rank}
rank = pd.concat(rank, axis=1)

table = pd.concat({"Unconditional Gen.": res_uncond, "Completion Gen.": res_completion, "Label-conditioned Gen.": res_cond}, axis=1)
table = table[["Label-conditioned Gen.", "Completion Gen.", "Unconditional Gen."]]

styler = table.style\
.background_gradient(cmap="Blues", gmap=rank, axis=None, high=0.5)\
.format("{:.2f}")
styler

In [ ]:
styler = res_completion.style\
.background_gradient(cmap="Blues", gmap=comp_rank, axis=None, high=0.5)\
.format("{:.2f}")
styler
print(styler.to_latex(convert_css=True, hrules=True, position_float="raggedright"))